## Set up

In [1]:
# TODO: 
# - Add x and y in a metatable
# - Create a list with all needed sites
# - Add soil texture to lookup table and save as csv
import os
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import pyarrow as pa
import pyarrow.parquet as pq

In [2]:
BASE_PATH = 'D:/Data/KIMoDIs/global-groundwater-models-main'

DATA_PATH = os.path.join(BASE_PATH, 'data')
MODEL_PATH = os.path.join(BASE_PATH, 'models')
RESULT_PATH = os.path.join(BASE_PATH, 'results')

In [3]:
# Old static features from Alex
static_df = pd.read_feather(os.path.join(DATA_PATH, 'static.feather'))

# Newly extracted
static_df_new = pq.read_table(os.path.join(DATA_PATH, "well_extracted_staticfeatures_all.parquet"))
static_df_new = static_df_new.to_pandas()
static_df_new = static_df_new[static_df_new['well_id'].isin(static_df['proj_id'])]
static_df_new.rename(columns={'well_id':'proj_id'}, inplace=True)

## Feature selection static features 

In [4]:
# Metadata:
# D:\Data\github\anyextract\data_proj
# list(static_df_new)

# eumohp, dsd, sd bis zur 6. Ordnung, 
# gwn_recharge, 
# hyraum_r, hyraum_gr, -> welchen?
# permeability coefficient (kf) -> überspannende Kategorien
# aquifer type (ha)
# soil type -> sehr viele Kategorien (557), erstmal nicht verwendet (buek_250bt)
# land_cover
# elevation
# topographischer Feuchteindex
# bodenart
# feldkapazität

# Subset
static_subset = static_df_new.filter(regex='proj_id|eumohp.+[0-6]|entgeom10kment|shannongeom10kmsha|unigeom10kmuni|gwn_recharge|hyraum_gr|huek250_kf|huek250_ha|probavlcclass|lai.*|twi|fk10dm1000_fc|boart1000_st')
list(static_subset)

['proj_id',
 'custom_twi',
 'probav_probavlcclass',
 'eumohp_dsd2',
 'eumohp_dsd3',
 'eumohp_dsd4',
 'eumohp_dsd5',
 'eumohp_dsd6',
 'eumohp_lp2',
 'eumohp_lp3',
 'eumohp_lp4',
 'eumohp_lp5',
 'eumohp_lp6',
 'eumohp_sd2',
 'eumohp_sd3',
 'eumohp_sd4',
 'eumohp_sd5',
 'eumohp_sd6',
 'gwn_recharge',
 'hyraum_gr',
 'copernicus_lai01',
 'copernicus_lai02',
 'copernicus_lai03',
 'copernicus_lai04',
 'copernicus_lai05',
 'copernicus_lai06',
 'copernicus_lai07',
 'copernicus_lai08',
 'copernicus_lai09',
 'copernicus_lai10',
 'copernicus_lai11',
 'copernicus_lai12',
 'fk10dm1000_fc',
 'amatulli_entgeom10kment',
 'amatulli_shannongeom10kmsha',
 'amatulli_unigeom10kmuni',
 'huek250_ha',
 'huek250_kf',
 'boart1000_st']

In [5]:
# MOHP is acutally 1-5
# Define a mapping to change MOHP names 
column_mapping_dsd = {f'eumohp_dsd{i}': f'eumohp_dsd{i-1}' for i in range(2, 7)} 
column_mapping_lp = {f'eumohp_lp{i}': f'eumohp_lp{i-1}' for i in range(2, 7)}
column_mapping_sd = {f'eumohp_sd{i}': f'eumohp_sd{i-1}' for i in range(2, 7)}
column_mapping = {**column_mapping_dsd, **column_mapping_lp, **column_mapping_sd}
column_mapping

# Rename the columns using the mapping
static_subset.rename(columns=column_mapping, inplace=True)

# Remaining columns with hard to understand column names
static_subset.rename(columns={'probav_probavlcclass':'land_cover', 
                              'huek250_ha':'aquifer_type', 
                              'huek250_kf':'permeability_coef', 
                              'boart1000_st': 'soil_texture', 
                              'fk10dm1000_fc': 'field_capacity',
                              'custom_twi': 'twi', 
                              'gwn_recharge': 'gw_recharge',
                              'amatulli_entgeom10kment': 'landform_entr10km', 
                              'amatulli_shannongeom10kmsha': 'landform_sha10km', 
                              'amatulli_unigeom10kmuni': 'landform_uni10km'}, inplace=True)

C:\Users\kunz.s\AppData\Local\Temp\16\ipykernel_37048\3346767697.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static_subset.rename(columns=column_mapping, inplace=True)
C:\Users\kunz.s\AppData\Local\Temp\16\ipykernel_37048\3346767697.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  static_subset.rename(columns={'probav_probavlcclass':'land_cover',


In [ ]:
static_subset.dtypes

In [7]:
# Rm where aquifer type, k.A. (kf == 0 does not exist)
static_subset = static_subset[static_subset['aquifer_type']!='k.A.']

In [8]:
# Kf überspannende Kategorien zusammenfassen
# static_subset[static_subset['permeability_coef']==0]
# 'hoch' didn't need to be aggregated with 'sehr hoch', because 'sehr hoch didn't exist
static_subset.loc[static_subset['permeability_coef'].isin(['3','4']), 'permeability_coef'] = '9'
static_subset.loc[static_subset['permeability_coef'].isin(['5','6','7']), 'permeability_coef'] = '10'

# Mäßig bis gering occurs 897 times and is an in-between category
# static_subset[static_subset['permeability_coef']=='0']

In [9]:
static_subset['aquifer_type'].unique()

['P', 'K/Ka', 'K', 'K/P', 'G']
Categories (6, object): ['G', 'K', 'k.A.', 'K/Ka', 'K/P', 'P']

In [27]:
# Land cover & soil texture should be categories
col_convert = ['land_cover', 'soil_texture']
static_subset[col_convert] = static_subset[col_convert].astype('category')

In [21]:
# Add elevation from original dataset
static_subset = static_subset.merge(static_df[['proj_id', 'elevation']], on='proj_id', how='left')

In [ ]:
# Checking for NAs
static_subset.isnull().any()

In [39]:
# One site doesn't have gwn recharge values
# RM for now
# static_subset[static_subset['gwn_recharge'].isnull()]
static_subset = static_subset[static_subset['gwn_recharge'].notnull()]

In [8]:
fc_missing = static_subset[static_subset['field_capacity'].isnull()][['proj_id', 'field_capacity']]
fc_missing = pa.Table.from_pandas(fc_missing)
pq.write_table(fc_missing, os.path.join(DATA_PATH, 'fc_missing.parquet'))

In [44]:
# 31 sites with no eumohp values
# RM for now
# static_subset[static_subset['eumohp_dsd1'].isnull()]['proj_id'].unique())
static_subset = static_subset[static_subset['eumohp_dsd1'].notnull()]

## Temporal features

### Temperature, Humidity & Precipitation

In [46]:
temporal_df_new = pq.read_table(os.path.join(DATA_PATH, "well_extracted_t_p_rh_1990-2020.parquet"))
temporal_df_new = temporal_df_new.to_pandas()
temporal_df_new.rename(columns={'well_id': 'proj_id', 
                                'date': 'time',
                                'p': 'precip',
                                'hurs': 'humid', 
                                'tas': 'temp'}, inplace=True)
temporal_df_new['time'] = pd.to_datetime(temporal_df_new['time'])

In [47]:
temporal_df_new[['precip', 'humid', 'temp']].describe()

,precip,humid,temp
count,1.519660e+08,1.519660e+08,1.519660e+08
mean,1.909476e+00,7.760586e+01,9.925549e+00
std,4.135705e+00,1.185131e+01,7.335859e+00
min,0.000000e+00,1.654206e+01,-2.050000e+01
25%,0.000000e+00,6.976090e+01,4.420087e+00
50%,1.000000e-01,7.913600e+01,1.000000e+01
75%,1.978661e+00,8.680000e+01,1.562035e+01
max,2.630464e+02,1.000000e+02,3.170000e+01


In [ ]:
# No NANs
temporal_df_new.isnull().any()

In [49]:
# Use data from 1990 until 2016
temporal_df_new = temporal_df_new[(temporal_df_new['time']>='1990-01-01') & (temporal_df_new['time']<='2016-01-01')]

In [50]:
# Aggregate from daily to weekly
temporal_df_new.set_index('time', inplace = True)
temporal_df_new = temporal_df_new.groupby('proj_id').resample('W')[['precip', 'humid', 'temp']].mean().reset_index()

### Leaf Area Index

In [51]:
# Merge the monthly values for each proj id:
# - lai values are transformed to long format 
# - a month column is created to merge the lai values
df_lai = static_df_new.filter(regex='proj_id|lai').melt(id_vars=['proj_id'], value_name='lai')

In [52]:
# Extract the month number from the 'variable' column 
df_lai['month'] = df_lai['variable'].str.extract(r'(\d+)').astype(int)
# Example
df_lai[df_lai['proj_id']=='HB_208']

,proj_id,variable,lai,month
0,HB_208,copernicus_lai01,0.341966,1
13475,HB_208,copernicus_lai02,0.201331,2
26950,HB_208,copernicus_lai03,0.323954,3
40425,HB_208,copernicus_lai04,0.580458,4
53900,HB_208,copernicus_lai05,0.973662,5
67375,HB_208,copernicus_lai06,1.164746,6
80850,HB_208,copernicus_lai07,1.154863,7
94325,HB_208,copernicus_lai08,0.946894,8
107800,HB_208,copernicus_lai09,0.648462,9
121275,HB_208,copernicus_lai10,0.365248,10


In [53]:
# No nan's
df_lai.isnull().any()

proj_id     False
variable    False
lai         False
month       False
dtype: bool

In [54]:
temporal_df_new['month'] = temporal_df_new['time'].dt.month
temporal_df_new = temporal_df_new.merge(df_lai[['proj_id', 'month', 'lai']], 
                                        on=['proj_id', 'month'], 
                                        how='left')

### Day as circular variable

In [55]:
# Encode day of the year as circular feature
temporal_df_new['day_sin'] = np.sin(2*np.pi / 365. * temporal_df_new['time'].dt.dayofyear).astype(np.float32)
temporal_df_new['day_cos'] = np.cos(2*np.pi / 365. * temporal_df_new['time'].dt.dayofyear).astype(np.float32)

## Merge spatial and temporal data

In [72]:
temporal_df_new.drop(['month'], axis=1, inplace=True)
# Some sites in temproal_df that are not in the static_features
temporal_df_new = temporal_df_new[temporal_df_new['proj_id'].isin(static_subset['proj_id'])]
features_df = temporal_df_new.merge(static_subset, on='proj_id',how='left')

In [74]:
# Rm lai columns
col_lai = list(features_df.filter(regex = 'copernicus.*'))
features_df.drop(col_lai, axis=1, inplace=True)

In [75]:
list(features_df)

['proj_id',
 'time',
 'precip',
 'humid',
 'temp',
 'lai',
 'day_sin',
 'day_cos',
 'twi',
 'land_cover',
 'eumohp_dsd1',
 'eumohp_dsd2',
 'eumohp_dsd3',
 'eumohp_dsd4',
 'eumohp_dsd5',
 'eumohp_lp1',
 'eumohp_lp2',
 'eumohp_lp3',
 'eumohp_lp4',
 'eumohp_lp5',
 'eumohp_sd1',
 'eumohp_sd2',
 'eumohp_sd3',
 'eumohp_sd4',
 'eumohp_sd5',
 'gwn_recharge',
 'hyraum_gr',
 'landform_entr10km',
 'landform_sha10km',
 'landform_uni10km',
 'aquifer_type',
 'permeability_coef',
 'soil_texture',
 'elevation']

In [77]:
features_df.isnull().any()

proj_id              False
time                 False
precip               False
humid                False
temp                 False
lai                  False
day_sin              False
day_cos              False
twi                  False
land_cover           False
eumohp_dsd1          False
eumohp_dsd2          False
eumohp_dsd3          False
eumohp_dsd4          False
eumohp_dsd5          False
eumohp_lp1           False
eumohp_lp2           False
eumohp_lp3           False
eumohp_lp4           False
eumohp_lp5           False
eumohp_sd1           False
eumohp_sd2           False
eumohp_sd3           False
eumohp_sd4           False
eumohp_sd5           False
gwn_recharge         False
hyraum_gr            False
landform_entr10km    False
landform_sha10km     False
landform_uni10km     False
aquifer_type         False
permeability_coef    False
soil_texture         False
elevation            False
dtype: bool

In [79]:
# Reorder columns
cols = ['proj_id',
     'time',
     'precip',
     'humid',
     'temp',
     'lai',
     'day_sin',
     'day_cos',
     'twi',
     'gw_recharge',
     'hyraum_gr',
     'aquifer_type',
     'permeability_coef',
     'soil_texture',
     'elevation',
     'land_cover',
     'landform_entr10km',
     'landform_sha10km',
     'landform_uni10km',
     'eumohp_dsd1',
     'eumohp_dsd2',
     'eumohp_dsd3',
     'eumohp_dsd4',
     'eumohp_dsd5',
     'eumohp_lp1',
     'eumohp_lp2',
     'eumohp_lp3',
     'eumohp_lp4',
     'eumohp_lp5',
     'eumohp_sd1',
     'eumohp_sd2',
     'eumohp_sd3',
     'eumohp_sd4',
     'eumohp_sd5']
features_df = features_df[cols]

In [80]:
# Save as parquet
table_features = pa.Table.from_pandas(features_df)
pq.write_table(table_features, os.path.join(DATA_PATH, 'features_df.parquet'))

## Lookup tables

In [ ]:
# Permeability coefficient (kf in m/s)
kf_lc = {'kf':[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 99], 
         'meaning': ['keine Angaben', 
                     'sehr hoch (>1E-2)',
                     'hoch >(1E-3 - 1E-2)',
                     'mittel (>1E-4 - 1E-3)',
                     'mäßig (1E-5 - 1E-4)',
                     'gering (>1E-7 - 1E-5)', 
                     'sehr gering (>1E-9 - 1E-7)', 
                     'äußerst gering (<1E-9)', 
                     'sehr hoch bis hoch (>1E-3)', 
                     'mittel bis mäßig (>1E-5 - 1E-3)', 
                     'gering bis äußerst gering (<1E-5)', 
                     'stark variabel', 
                     'mäßig bis gering (>1E-6 - 1E-4)', 
                     'Gewässer']}
kf_lc = pd.DataFrame(data=kf_lc)

# Aquifer type
aquifertype_lc = {'code':[None, 1, 2, 3, 4, 5], 
                  'abbrev':['k.A', 'P', 'K/P', 'K', 'K/KA', 'G'] ,
                  'aq_type':['keine Angaben', 'Poren', 'Kluft/Poren', 'Kluft', 'Kluft/Karst', 'Gewässer']}
aquifertype_lc = pd.DataFrame(data = aquifertype_lc)

# Hyraum
hyraum_lc = {'code':[1, 11, 12, 13, 14, 15, 16, 17, 
                     2, 21, 22, 23,
                     3, 31, 32, 33, 
                     4, 41,
                     5, 51, 52, 53, 54, 
                     6, 61, 62, 63, 64, 65, 66,
                     7, 71, 
                     8, 81, 82, 83, 
                     9, 91, 92, 93, 94, 95, 96, 97,
                     10, 101], 
             'hyraum':['Nord- und mitteldt. Lockergesteinsgebiet', 'Nordseeinseln und Watten', 'Nordseemarschen', 'Niederungen im nord-und mitteldt. Lockergesteinsgebiet', 'Norddeutsches Jungpleistozän', 'Nord- und mitteldt. Mittelpleistozän', 'Altmoränengeest', 'Lausistzer Känozoikum',
                       'Rheinisch-Westfälisches Tiefland', 'Sandmünsterland', 'Münsterländer Kreidebecken', 'Niederrheinische Tieflandsbucht', 
                       'Oberrheingraben mit Mainzer Becken und nordhessischem Tertiär', 'Oberrheingraben mit Mainzer Becken', 'Untermainsenke', 'Nordhessisches Tertiär', 
                       'Alpenvorland', 'Süddeutsches Molassebecken', 
                       'Mitteldeutsches Bruchschollenland', 'Nordwestdeutsches Bergland', 'Mitteldeutscher Buntsandstein', 'Subherzyne Senke', 'Thüringische Senke', 
                       'West- und süddeutsches Schichtstufen- und Bruchschollenland', 'Südwestdeutsche Trias', 'Süddeutscher Buntsandstein und Muschelkalk', 'Süddeutscher Keuper und Albvorland', 'Schwäbische und Fränkische Alb', 'Nördlinger Ries', ' Thüringisch-Fränkisches Bruchschollenland',
                       'Alpen', 'Nordalpen', 
                       'West- und mitteldeutsches Grundgebirge', ' Rheinisches Schiefergebirge', ' Saar-Nahe-Becken', 'Mitteldeutsches Grundgebirge', 
                       'Südostdeutsches Grundgebirge', 'Elbtalgraben', 'Fichtelgebirge-Erzgebirge', 'Lausitzer Granodioritkomplex', 'Nordwestsächsische Senke', 'Oberpfälzer-Bayrischer Wald', 'Südostdeutsches Schiefergebirge', 'Thüringer Wald', 
                       'Südwestdeutsches Grundgebirge', 'Schwarzwald, Vorspeesart und Odenwald']}
hyraum_lc = pd.DataFrame(data = hyraum_lc)